In [ ]:
# CRIA O AMBIENTE VIRTUAL

conda create -n PEFT_PromptTuning python=3.11
conda activate PEFT_PromptTuning

In [ ]:
# INSTALA AS DEPENDENCIAS

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install tqdm ipykernel ipywidgets transformers==4.42.4 datasets==2.19.1 peft==0.10.0 trl==0.9.4
# pip installaccelerate==0.30.1 tokenizers==0.19.1 bitsandbytes==0.43.1

python -m ipykernel install --user --name=PEFT_PromptTuning --display-name="PEFT_PromptTuning"

In [ ]:
# VERIFICAÇÃO DO DATASET

import pandas as pd

df = pd.read_json("dataset_classificacao_juridica.jsonl", lines=True)
print(df['area'].value_counts())

In [1]:
# IMPORTA AS BIBLIOTECAS

import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import PeftModel, get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
from trl import SFTTrainer
import numpy as np

In [2]:
# CONFIGURAÇÕES

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_FILE = "dataset_classificacao_juridica.jsonl"
OUTPUT_DIR = "./modelo_juridico_prompt_tuning"

In [3]:
# CARREGAR DATASET
full_dataset = load_dataset('json', data_files={'train': DATASET_FILE}, split='train')
train_test_split = full_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
test_validation_split = train_test_split['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

final_datasets = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_validation_split['train'],
    'test': test_validation_split['test']
})

print("Dataset carregado e dividido:")
print(final_datasets)

# Função para formatar cada exemplo como uma instrução e resposta em texto
def formatar_para_geracao_de_classe(examplo):
    return [
        f"### Instruction:\nClassifique o seguinte texto na área do direito correspondente:\n\n{examplo['texto']}\n\n### Response:\n{examplo['area']}{tokenizer.eos_token}"
    ]

Dataset carregado e dividido:
DatasetDict({
    train: Dataset({
        features: ['texto', 'area'],
        num_rows: 817
    })
    validation: Dataset({
        features: ['texto', 'area'],
        num_rows: 102
    })
    test: Dataset({
        features: ['texto', 'area'],
        num_rows: 103
    })
})


In [6]:
# CARREGAR TOKENIZADOR E MODELO

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
)
model.config.pad_token_id = tokenizer.pad_token_id

peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,  # Tarefa de Classificação de Sequência
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=200,  # Tamanho do prompt virtual que será treinado
    prompt_tuning_init_text="Classifique o seguinte texto jurídico na área do direito correspondente:",
    tokenizer_name_or_path=MODEL_NAME,
)

model.config.pad_token_id = tokenizer.eos_token_id

# Adiciona o adaptador de Prompt Tuning ao modelo
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


trainable params: 44,800 || all params: 494,077,568 || trainable%: 0.00906740214524372


In [10]:
# EXECUTA O TREINAMENTO

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-5,
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    overwrite_output_dir=True,
    report_to="none",
    lr_scheduler_type='cosine',
    warmup_steps=100,
    bf16=True,
)

# Instanciar o Trainer padrão
trainer = SFTTrainer(
    model=model,
    args=training_args,
    peft_config=peft_config,
    train_dataset=final_datasets["train"],
    eval_dataset=final_datasets["validation"],
    tokenizer=tokenizer,
    formatting_func=formatar_para_geracao_de_classe,
    max_seq_length=1024,
)

trainer.train()

print(f"Treinamento concluído. Salvando o adaptador em: {OUTPUT_DIR}")
trainer.save_model()

/home/ricardo/.conda/envs/PEFT_PromptTuning/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ricardo/.conda/envs/PEFT_PromptTuning/lib/python3.11/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/ricardo/.conda/envs/PEFT_PromptTuning/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.706446
2,No log,2.706239
3,No log,2.706094
4,No log,2.706033
5,No log,2.706822
6,No log,2.706104
7,No log,2.705896
8,No log,2.706474
9,No log,2.705169
10,No log,2.705951


Treinamento concluído. Salvando o adaptador em: ./modelo_juridico_prompt_tuning


In [ ]:
# VERIFICAÇÃO E DEMONSTRAÇÃO

import torch
import torch.nn.functional as F
from peft import PeftModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)

model_com_prompt = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_com_prompt.config.pad_token_id = tokenizer.eos_token_id

# Garante que o modelo esteja em modo de avaliação (desativa dropout, etc.)
model_com_prompt.eval()

# Move o modelo para a GPU, se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_com_prompt.to(device)


def classificar_texto(texto):
    # Tokeniza a entrada, convertendo para tensores PyTorch e movendo para a GPU
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True).to(device)

    # Faz a predição sem calcular gradientes (mais rápido e economiza memória)
    with torch.no_grad():
        outputs = model_com_prompt(**inputs)

    # A saída do modelo são 'logits' (valores brutos)
    logits = outputs.logits

    # Aplica a função softmax para converter logits em probabilidades
    probabilities = F.softmax(logits, dim=-1)

    # Pega o índice da classe com a maior probabilidade
    predicted_class_id = torch.argmax(probabilities, dim=-1).item()

    # Usa o mapeamento id2label para obter o nome da classe
    predicted_class_label = model_com_prompt.config.id2label[predicted_class_id]
    
    # Pega a probabilidade da classe prevista
    confidence_score = probabilities[0][predicted_class_id].item()

    return {"label": predicted_class_label, "score": confidence_score}


texto_exemplo = "O réu foi condenado por homicídio qualificado, com pena base acima do mínimo legal."
resultado = classificar_texto(texto_exemplo)

print("\n--- Teste de Inferência Manual ---")
print(f"Texto: '{texto_exemplo}'")
print(f"Resultado da Classificação: {resultado}")

texto_exemplo_2 = "A empresa foi autuada por não recolher o PIS e a COFINS sobre o faturamento."
resultado_2 = classificar_texto(texto_exemplo_2)
print(f"\nTexto: '{texto_exemplo_2}'")
print(f"Resultado da Classificação: {resultado_2}")